In [2]:
import math
import torch

In [4]:
batch_size = 1  # batch为多少条数据 
length = 4      # length 为 4

x = torch.randn(batch_size, length, 512) #input :  batch_size, length, embd_dim
y = torch.randint(low=0, high=32000, size=(batch_size, length), dtype=torch.long)

print(x.shape)
print(y.shape)
print(y)

torch.Size([1, 4, 512])
torch.Size([1, 4])
tensor([[   44, 23750, 14680, 24802]])


In [5]:
# attn
q = torch.randn(512, 512)  
k = torch.randn(512, 512)
v = torch.randn(512, 512)
o = torch.randn(512, 512)

mask=torch.tril(torch.ones(1, 4, 4))
print(mask)

# scaled dot produc attention 
Q,K,V = x@q, x@k, x@v 
scores = Q@K.transpose(1,2) / math.sqrt(512.0)
scores = scores.masked_fill(mask == 0, float('-inf'))
weight = torch.nn.functional.softmax(scores, dim=2)
attn = weight@V
attn = attn@o
attn.shape

tensor([[[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]]])


torch.Size([1, 4, 512])

In [6]:
# mlp
mlp_up = torch.randn(512, 1024)
mlp_down = torch.randn(1024, 512)
mlp = attn @ mlp_up @ mlp_down
mlp.shape

torch.Size([1, 4, 512])

In [7]:
# output
lm_head = torch.randn(512, 32000) 
logits = mlp@lm_head
logits.shape

torch.Size([1, 4, 32000])

In [11]:
# loss
# probs
probs = torch.softmax(logits, dim=2) # 4个token 对应 4个logits[32000]
print(probs.shape) # model ouput prob
print(y)    # model lables   # 1logits 32000 <-> label 1 
# label 有4个

# Loss
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(probs.view(-1,32000), y.view(-1)) #4token  4 CE loss
print(loss)

# pred
pred = torch.argmax(probs, dim=2) # generation 
print(pred) # model pred     4 pred
print(pred[0,-1]) # model pred    推理的next token预测
# greedy 贪婪的解码

torch.Size([1, 4, 32000])
tensor([[   44, 23750, 14680, 24802]])
tensor(10.3736)
tensor([[17211, 17211, 22707, 17211]])
tensor(17211)
